## 研究の流れ

このipynb と README.md を見るだけで流れがわかるようにしました。  
なお、各処理の詳細が知りたい場合は各ファイルを閲覧してください。


In [ ]:
# 標準モジュールのインポート
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import subprocess
from glob import glob

# src配下のファイルを読み込めるようにする
sys.path.append(os.getcwd() + "\src")


In [ ]:
# 共通する自作モジュールのインポート
from Visualization.SnapData import SnapData
from SetLogger import logger_conf
logger = logger_conf()


In [ ]:
# ディレクトリの作成と定義
from params import ROOT_DIR, SNAP_PATH
data_dir = ROOT_DIR + "\\data"

# 出力先フォルダの作成
subprocess.run([ROOT_DIR+"\etc\mkdir.bat"])


### 1. データの加工

##### 1-1 元データを各種パラメータに分割する

In [ ]:
from Processing.separater_loop import data_processing
data_processing()


##### 1-2 バイナリを .npy に変換

In [ ]:
import os
import sys
sys.path.append(os.getcwd() + "\src")

from params import SNAP_PATH
from Processing.snap2npy import snap2npy


In [ ]:
sp = SnapData()

for i in [77, 497, 4949]:
    logger.debug("START", extra={"addinfo": f"Snap{i}"})

    for target in ["density", "enstrophy", "magfieldx", "magfieldy", "magfieldz", "pressure", "velocityx", "velocityy", "velocityz"]:
        logger.debug("START", extra={"addinfo": f"{target}"})

        for path in glob(SNAP_PATH + f"\snap{i}\{target}\*\*"):
            # print(path)
            snap2npy(sp, path, i)

        logger.debug("END", extra={"addinfo": f"{target}"})
    logger.debug("END", extra={"addinfo": f"Snap{i}"})


## 2. 可視化

#### 2-1 各種可視化

In [ ]:
# 標準モジュールのインポート
import os
import sys
from glob import glob

# src配下のファイルを読み込めるようにする
sys.path.append(os.getcwd() + "\src")

from params import SNAP_PATH
from Visualization.Visualize import VisualizeMethod


In [ ]:
for i in [77, 497, 4949]:
    target_path = SNAP_PATH + f"\snap{i}"
    viz = VisualizeMethod()

    files = {}
    files["density"] = glob(target_path + f"\density\*\*")
    files["velocityx"] = glob(target_path + f"\\velocityx\*\*")
    files["velocityy"] = glob(target_path + f"\\velocityy\*\*")
    for dens_path, vx_path, vy_path in zip(files["density"], files["velocityx"], files["velocityy"]):
        viz.drawEnergy_for_velocity(dens_path, vx_path, vy_path)

    files["magfieldx"] = glob(target_path + f"\magfieldx\*\*")
    files["magfieldy"] = glob(target_path + f"\magfieldy\*\*")
    for magx_path, magy_path in zip(files["magfieldx"], files["magfieldy"]):
        viz.drawEnergy_for_magfield(magx_path, magy_path)
        
    files["enstrophy"] = glob(target_path + f"\enstrophy\*\*")
    for target in ["velocityx", "velocityy", "magfieldx", "magfieldy", "density", "enstrophy"]:
        for path in files[target]:
            viz.drawHeatmap(path)
            viz.drawEdge(path)


#### 2-2 AVS

#### 2-3 StreamLines

In [ ]:
from StreamLines.StreamLine import ohno_stream


#### 2-4 LIC

In [ ]:
import os
import sys
from glob import glob
sys.path.append(os.getcwd() + "\src")

from params import SRC_PATH, SNAP_PATH, IMGOUT
from LIC.LIC import LicMethod


In [ ]:
logger.debug("START", extra={"addinfo": "処理開始"})

lic = LicMethod()
numbers  = [77, 497, 4949]
out_dir = IMGOUT + "\LIC"
lic.makedir("\LIC")

for i in numbers:
    indir = SNAP_PATH + f"\half\snap{i}"
    dir_basename = os.path.basename(indir) # snap77
    base_out_path = out_dir + "\\" + os.path.basename(indir) # .\imgout\LIC\snap77
    lic.makedir(f"\LIC\snap{i}")

    binary_paths = glob(indir+"\magfieldx\*\*.npy")
    # ファイルが無い場合
    if binary_paths == []:
        raise "Error File not Found"
    
    for xfile in binary_paths[-1:]:
        yfile = xfile.replace("magfieldx", "magfieldy")
        out_path = base_out_path + f"\lic_{dir_basename}.{os.path.splitext(os.path.basename(xfile))[0]}.bmp"
        # print(out_path) # .\imgout\LIC\snap77\lic_snap77.magfieldx.01.14.bmp
        
        command = lic.set_command(xfile, yfile, out_path)
        lic.LIC(command)


### 3. 機械学習

In [ ]:
# 機械学習関係のインポート


#### 3-1 教師データの作成

#### 3-2 SVM

#### 3-3 非線形SVM

#### 3-4 k-Means

In [ ]:
import os
import sys
import numpy as np
from glob import glob
sys.path.append(os.getcwd() + "\src")

from params import SNAP_PATH
from k_means.Clustering import ClusteringMethod


In [ ]:
cluster = ClusteringMethod()

for num in [77, 497, 4949]:
    for target in ["enstrophy"]:
        path_list = glob(SNAP_PATH + f"/snap{num}/{target}/*/*")
        num_of_data = len(path_list) # リコネクションがない画像の枚数

        temp_data = cluster.compress(cluster.loadSnapData(path_list[0],z=3))
        IMGSHAPE = temp_data.shape

        N_col = IMGSHAPE[0] * IMGSHAPE[1] * 1 # 行列の列数
        X_train = np.zeros((num_of_data, N_col)) # 学習データ格納のためゼロ行列生成
        y_train = np.zeros((num_of_data)) # 学習データに対するラベルを格納するためのゼロ行列生成

        # リコネクションがない画像を行列に読み込む
        for idx, item in enumerate(path_list[:10]):
            X_train[idx, :] = cluster.load_regularize(item)
            y_train[idx] = 0 # リコネクションがないことを表すラベル

        X_train_pca = cluster.PCA(X_train)
        cluster_labels = cluster.KMeans(X_train_pca)
        df_re = cluster.save_result(cluster_labels, path_list)
        # display(df_re)


#### 3-5 XGBoost